In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
!unzip '/content/drive/My Drive/Colab Notebooks/1332faces.zip' -d '/content/imgData'

In [3]:
from PIL import Image
import numpy as np
import glob

In [32]:
cwd = '/content/imgData/dataset'
image_list_part1 = []
data_list = []
# for filename in glob.glob(cwd + "//0001-3000//*.jpg"):
#     im = Image.open(filename)
#     image_list_part1.append(im)
#     print(im.size)
#     data_list.append(np.asarray(im))

# image_list_part2 = []
# for filename in glob.glob(cwd + "//3001-6000//*.jpg"):
#     im = Image.open(filename)
#     image_list_part2.append(im)
#     print(im.size)
#     data_list.append(np.asarray(im))

# image_list_part3 = []
# for filename in glob.glob(cwd + "//6001-9000//*.jpg"):
#     im = Image.open(filename)
#     image_list_part3.append(im)
#     print(im.size)
#     data_list.append(np.asarray(im))

# image_list_part4 = []
# for filename in glob.glob(cwd + "//9001-13322//*.jpg"):
#     im = Image.open(filename)
#     image_list_part4.append(im)
#     print(im.size)
#     data_list.append(np.asarray(im))

In [34]:

# You just need to compare the aspect ratios - depending on which is larger, that will tell you whether to chop off the sides or the top and bottom. e.g. how about:

def resizeImg(image, ideal_width, ideal_height):
  width  = image.size[0]
  height = image.size[1]
  aspect = width / float(height)
  ideal_aspect = ideal_width / float(ideal_height)
  if aspect > ideal_aspect:
      # Then crop the left and right edges:
      new_width = int(ideal_aspect * height)
      offset = (width - new_width) / 2
      resize = (offset, 0, width - offset, height)
  else:
      # ... crop the top and bottom:
      new_height = int(width / ideal_aspect)
      offset = (height - new_height) / 2
      resize = (0, offset, width, height - offset)
  thumb = image.crop(resize).resize((ideal_width, ideal_height), Image.ANTIALIAS)
  return thumb

In [45]:
print(len(data_list))
testCat = Image.open(cwd + "//testCat.jpg")
exampleImage = resizeImg(image = testCat, ideal_width = 64, ideal_height = 64)
print(exampleImage)

1
<PIL.Image.Image image mode=RGB size=64x64 at 0x7F67CE42F208>


In [46]:
print(exampleImage)
print(type(exampleImage))

image_sequence = exampleImage.getdata()
image_array = np.array(image_sequence)

<PIL.Image.Image image mode=RGB size=64x64 at 0x7F67CE42F208>
<class 'PIL.Image.Image'>


In [47]:
print(type(image_array))
print(image_array.shape)

<class 'numpy.ndarray'>
(4096, 3)


In [39]:
trainX = []
for pixels in data_list:
  print('Min: %.3f, Max: %.3f' % (pixels.min(), pixels.max()))
  # convert from integers to floats
  pixels = pixels.astype('float32')
  # normalize to the range 0-1
  pixels /= 255.0
  # confirm the normalization
  print('Min: %.3f, Max: %.3f' % (pixels.min(), pixels.max()))
  trainX.append(pixels)


AttributeError: ignored

In [30]:
i = 1
print(type(trainX[i]))
print(len(trainX[i]))
print(len(trainX[i][i]))

<class 'numpy.ndarray'>
398
309


In [22]:
print(trainX[0][0])

[[0.         0.         0.01568628]
 [0.         0.         0.01568628]
 [0.         0.         0.01568628]
 [0.         0.         0.01568628]
 [0.         0.         0.01568628]
 [0.         0.         0.01568628]
 [0.         0.         0.01568628]
 [0.         0.         0.01568628]
 [0.         0.         0.01568628]
 [0.         0.         0.01568628]
 [0.         0.         0.01568628]
 [0.         0.         0.01568628]
 [0.         0.         0.01568628]
 [0.         0.         0.01568628]
 [0.         0.         0.01568628]
 [0.         0.         0.01568628]
 [0.         0.         0.        ]
 [0.         0.         0.        ]
 [0.         0.         0.        ]
 [0.         0.         0.        ]
 [0.         0.         0.        ]
 [0.         0.         0.        ]
 [0.         0.         0.        ]
 [0.         0.         0.        ]
 [0.         0.         0.        ]
 [0.         0.         0.        ]
 [0.         0.         0.        ]
 [0.         0.         0.  

In [9]:
from sklearn import decomposition
from sklearn.cluster import MiniBatchKMeans
from sklearn.datasets import fetch_olivetti_faces
import matplotlib.pyplot as plt
from numpy.random import RandomState
from time import time
import logging

In [11]:
faces, _ = fetch_olivetti_faces(return_X_y=True, shuffle=True,random_state=RandomState(0))
n_samples, n_features = faces.shape

downloading Olivetti faces from https://ndownloader.figshare.com/files/5976027 to /root/scikit_learn_data


In [24]:
print(type(faces))
print(len(faces[0]))
print(faces[0])

<class 'numpy.ndarray'>
4096
[0.6694215  0.6363636  0.6487603  ... 0.08677686 0.08264463 0.07438017]


In [37]:
# Display progress logs on stdout
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s %(levelname)s %(message)s')

In [41]:
n_row, n_col = 2, 3
n_components = n_row * n_col
image_shape = (64, 64)
rng = RandomState(0)
faces, _ = fetch_olivetti_faces(return_X_y=True, shuffle=True,random_state=rng)
n_samples, n_features = faces.shape

downloading Olivetti faces from https://ndownloader.figshare.com/files/5976027 to /root/scikit_learn_data


In [55]:
faces[0]

array([0.6694215 , 0.6363636 , 0.6487603 , ..., 0.08677686, 0.08264463,
       0.07438017], dtype=float32)